# Анализ сайта gismeteo.ru

Для выполнения итогового задания в качестве источника данных метеосводок был выбран сервис Дневник сайта Gismeteo.ru, в котором можно бесплатно получить исторические данные о погоде, начиная с 1997 года. Необходимо получить данные с 2008 по 2018 года включительно (последний год необходим для прогнозирования навигации на сезон).

todo: добавить список коммерческих ресурсов, с которых можно получить исторические данные и прогнозы на долгий срок

![Внешний вид сервиса Дневник от gismeteo](pictures/gismeteo_example.png)

После исполнения скрипта на загрузку данных из сайта автоматизированной информационной системы государственного мониторинга водных объектов Российской Федерации (далее - "АИС ГМВО") **src\preparation\scrape_water_level.py**, в **data\raw\water_posts_data.json** будет находиться словарь **{uid: наименование поста наблюдения}**. Наименования представлены в виде: *название водного объекта* - *название населённого пункта*

## Получение списка населённых пунктов с постами наблюдения

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка списка постов

In [16]:
from src.utils import *

if not is_data_exists(DATA_POSTS_RAW, is_raw=True):
    print('Данные с постами наблюдения АИС ГМВО не были загружены. Пожалуйста, '
          'выполните скрипт src\preparation\scrape_water_level.py'
          'и попробуйте ещё раз.')
else:
    file_posts = open_file(DATA_POSTS_RAW, is_raw=True)
    posts = json.loads(file_posts)
posts

{'09386': 'р.Подкаменная Тунгуска - пос.Чемдальск',
 '09387': 'р.Подкаменная Тунгуска - с.Ванавара',
 '09388': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
 '09389': 'р.Подкаменная Тунгуска - с.Байкит',
 '09390': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
 '09392': 'р.Чуня - пос.Стрелка Чуня',
 '09393': 'р.Чуня - пос.Муторай',
 '09396': 'руч.Миханский - пос.Вельмо 2-е',
 '09397': 'р.Тея - пгт Тея',
 '09499': 'р.Бурная - пос.Бурный',
 '09523': 'р.Подкаменная Тунгуска - пос.Суломай',
 '09560': 'р.Вельмо - пос.Светлана',
 '09568': 'р.Вельмо - пос.Вельмо 2-е',
 '09403': 'р.Нижняя Тунгуска - д.Верхнекарелина',
 '09404': 'р.Нижняя Тунгуска - с.Подволошино',
 '09405': 'р.Нижняя Тунгуска - с.Преображенка',
 '09406': 'р.Нижняя Тунгуска - с.Ербогачен',
 '09408': 'р.Нижняя Тунгуска - пос.Кислокан',
 '09410': 'р.Нижняя Тунгуска - пгт Тура',
 '09413': 'р.Нижняя Тунгуска - пос.Учами',
 '09415': 'р.Нижняя Тунгуска - факт.Большой Порог',
 '09416': 'р.Непа - с.Токма',
 '09417': 'р.Непа - д.Ика',
 '09419'

### Создание набора с очищенными названиями населённых пунктов

In [4]:
#test_name = next(iter(posts.values()))  # р.Подкаменная Тунгуска - пос.Чемдальск
#test_name = posts['09388']  # р.Подкаменная Тунгуска - факт.Усть-Камо
test_name = posts['09415']  # р.Нижняя Тунгуска - факт.Большой Порог
test_name

'р.Нижняя Тунгуска - факт.Большой Порог'

In [5]:
import re

'''
Между видом и названием населённого пункта есть или точка, или пробел, используем [ \.]{1}
Учтено наличие в названии пункта пробела и тире
capitalise() делается для того, чтобы привести названия из двух слов к единому формату, пример: Большой Порог и Большой порог
В БД gismeteo названия некоторых пунктов отличаются от таковых из сайта АИС ГМВО
'''
def clear_name(input_str):
    result = re.search(r" - \w+[ \.]{1}([\w -]+)", input_str).group(1)
    result = result.replace('Вельмо 2-е', 'Вельмо')
    result = result.replace('Стрелка чуня', 'Стрелка-Чуня')
    return result.capitalize()

clear_name(test_name)

'Большой порог'

In [6]:
for uid, name in posts.items():
    posts[uid] = {'name': name, 'clean_name': clear_name(name)}

posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск'},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара'},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-камо'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит'},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка'},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня', 'clean_name': 'Стрелка чуня'},
 '09393': {'name': 'р.Чуня - пос.Муторай', 'clean_name': 'Муторай'},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е', 'clean_name': 'Вельмо'},
 '09397': {'name': 'р.Тея - пгт Тея', 'clean_name': 'Тея'},
 '09499': {'name': 'р.Бурная - пос.Бурный', 'clean_name': 'Бурный'},
 '09523': {'name': 'р.Подкаменная Тунгуска - пос.Суломай',
  'clean_name': 'Суломай'},
 '09560': {'name': 'р.Вельмо - пос.Светлана', 'clean_name': 'Светлана'},
 '09568': {'name': 'р.Вельмо - пос.Вельм

### Создание списка с уникальными населёнными пунктами

In [7]:
names = set([x['clean_name'] for x in posts.values()])
names

{'Байкит',
 'Большой порог',
 'Бурный',
 'Ванавара',
 'Вельмо',
 'Верхнекарелина',
 'Ербогачен',
 'Ика',
 'Кербо',
 'Кислокан',
 'Кузьмовка',
 'Муторай',
 'Непа',
 'Подволошино',
 'Преображенка',
 'Светлана',
 'Стрелка чуня',
 'Суломай',
 'Тембенчи',
 'Тея',
 'Токма',
 'Тура',
 'Усть-камо',
 'Учами',
 'Чемдальск'}

In [8]:
print(f'Число постов: {len(posts)}\nЧисло уникальных пунктов: {len(names)}')

Число постов: 28
Число уникальных пунктов: 25


## Проверка наличия прогноза погоды

С получением метео-данных есть 2 проблемы:
1. Не все населённые пункты представлены в списке городов.
2. В некоторых пунтах за период с 2008 по 2018 не велись наблюдения метео-данных. При попытке их получить сайт показывает сообщение:
> Наблюдения метео-данных в данный период не велись.

![Сообщение об ошибке](pictures/gismeteo_error.png)

Необходимо обнаружить проблемные населённые пункты и с использованием карт найти ближайшие пункты, по которым есть необходимые данные.

### Получение списка населённых пунктов с gismeteo

После открытия https://www.gismeteo.ru/diary/ и выбора страны Россия и области Красноярский край происходит 3 GET запроса:
![GET запросы](pictures/gismeteo_requests.png)

1. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/countries/?fr=sel  
Получение списка стран c ID и названиями.

2. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/districts/?country=156&fr=sel  
Получения списка областей c ID и названиями. В параметре country указан ID выбранной страны (156 - Россия).

3. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district=301&fr=sel  
Получения списка городов c ID и названиями. В параметре district указан ID выбранной области (301 - Красноярский край).

Во всех запросах **63466572668a39754a9ddf4c8b3437b0** - это стандартный api-токен, который не зависит от IP-адреса пользователя. Запросы возвращают данные в формате xml.

In [9]:
from src.preparation.utils import get_url
import xmltodict

# если данные ранее были загружены, то используем их
if is_data_exists(DATA_GISMETEO_CITIES, is_raw=True):
    cities = open_file(DATA_GISMETEO_CITIES, is_raw=True)
else:
    # необходимо имитировать браузер, иначе соединение будет прервано
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'}
    url = 'https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district=301&fr=sel'
    r = get_url(url, headers=headers)
    cities = r.text
    write_data(DATA_GISMETEO_CITIES, data=cities, is_raw=True)

cities = xmltodict.parse(cities)  # конвертация xml в dict
print(type(cities), f'keys of dict: {cities.keys()}')
cities

<class 'collections.OrderedDict'> keys of dict: odict_keys(['document'])


OrderedDict([('document',
              OrderedDict([('item',
                            [OrderedDict([('@id', '157182'),
                                          ('@n', 'Абакумовка'),
                                          ('@country_id', '156'),
                                          ('@country_name', 'Россия'),
                                          ('@district_id', '301'),
                                          ('@district_name',
                                           'Красноярский край'),
                                          ('@kind', 'T')]),
                             OrderedDict([('@id', '157882'),
                                          ('@n', 'Абакшино'),
                                          ('@country_id', '156'),
                                          ('@country_name', 'Россия'),
                                          ('@district_id', '301'),
                                          ('@district_name',
                                   

In [10]:
# создадим новый словарь id: название на основе полученных данных
cities_gismeteo = []

for city in cities['document']['item']:
    city_id = city['@id']
    name = city['@n']
    if name in names:
        cities_gismeteo.append({'city_id': city_id, 'name': name})

cities_gismeteo

[{'city_id': '3995', 'name': 'Байкит'},
 {'city_id': '158140', 'name': 'Бурный'},
 {'city_id': '4036', 'name': 'Ванавара'},
 {'city_id': '157875', 'name': 'Вельмо'},
 {'city_id': '158142', 'name': 'Кислокан'},
 {'city_id': '158143', 'name': 'Кузьмовка'},
 {'city_id': '158146', 'name': 'Муторай'},
 {'city_id': '157247', 'name': 'Преображенка'},
 {'city_id': '156730', 'name': 'Преображенка'},
 {'city_id': '158152', 'name': 'Суломай'},
 {'city_id': '157881', 'name': 'Тея'},
 {'city_id': '4015', 'name': 'Тура'},
 {'city_id': '158154', 'name': 'Учами'},
 {'city_id': '158155', 'name': 'Чемдальск'}]

В списке есть дубликат населённого пункта Преображенка.

### Реализация проверки наличия прогноза

Получение исторических метео-данных происходит следующим образом:
1. Задаются все необходимые параметры - страна, область, город, месяц и год
2. Нажатие на кнопку Получить дневник выполняет перенаправление на url вида  
https://www.gismeteo.ru/diary/4036/2008/1/  
в котором:  
- **4036** - ID населённого пункта
- **2008** - год
- **1** - месяц без лидирующего нуля

Если данные погоде не были зафиксированы, то вместо дневника будет сообщение об отсутствии данных.

In [ ]:
def get_gismeteo_page(city_id, year, month):
    pass